In [8]:
import pandas as pd
import numpy as np
from scipy.stats import friedmanchisquare, wilcoxon
import scikit_posthocs as sp
import matplotlib.pyplot as plt
import seaborn as sns

# Step 1: Read the CSV files
xgboost_df = pd.read_csv('21.csv')
pr_df = pd.read_csv('4_degree.csv')
ann_df = pd.read_csv('metrics_128_units.csv')

# Step 2: Extract the performance metrics for the folds (excluding mean, std, sum)
xgboost_folds = xgboost_df.iloc[0:5].reset_index(drop=True)
pr_folds = pr_df.iloc[0:5].reset_index(drop=True)
ann_folds = ann_df.iloc[0:5].reset_index(drop=True)

# Define the list of metrics to analyze
metrics = ['mae', 'mse', 'rmse', 'r2', 'training_time', 'inference_time']

# Ensure that the metrics columns exist in all DataFrames
for df in [xgboost_df, pr_df, ann_df]:
    missing_metrics = set(metrics) - set(df.columns)
    if missing_metrics:
        print(f"Error: Missing columns {missing_metrics} in DataFrame.")
        exit()

# Step 3: Create a DataFrame for mean performance metrics
mean_metrics_df = pd.DataFrame({
    'Metric': metrics,
    'XGBoost': xgboost_df.iloc[5][metrics].values,
    'PR': pr_df.iloc[5][metrics].values,
    'ANN': ann_df.iloc[5][metrics].values
})

print('Mean Performance Metrics:')
print(mean_metrics_df)

# Save the mean_metrics DataFrame to a CSV file
mean_metrics_df.to_csv('mean_performance_metrics.csv', index=False)

# Step 4: Perform statistical tests for each metric
# Initialize a list to store Friedman test results
friedman_results = []

# Initialize a list to store pairwise Wilcoxon test results
pairwise_results_list = []

for metric in metrics:
    # Prepare data for the current metric
    data = pd.DataFrame({
        'XGBoost': xgboost_folds[metric],
        'PR': pr_folds[metric],
        'ANN': ann_folds[metric]
    })
    
    # Check if all values are numeric
    if not np.issubdtype(data['XGBoost'].dtype, np.number) or \
       not np.issubdtype(data['PR'].dtype, np.number) or \
       not np.issubdtype(data['ANN'].dtype, np.number):
        print(f"Error: Non-numeric data found in metric '{metric}'.")
        continue  # Skip this metric
    
    # Perform the Friedman test
    statistic, p_value = friedmanchisquare(data['XGBoost'], data['PR'], data['ANN'])
    friedman_results.append({'Metric': metric, 'Statistic': statistic, 'p-value': p_value})
    
    # Perform Nemenyi post-hoc test if Friedman test is significant
    if p_value < 0.05:
        nemenyi_result = sp.posthoc_nemenyi_friedman(data.values)
        nemenyi_result.columns = ['XGBoost', 'PR', 'ANN']
        nemenyi_result.index = ['XGBoost', 'PR', 'ANN']
        print(f'\nNemenyi post-hoc test results for {metric}:')
        print(nemenyi_result)
    
    # Perform pairwise Wilcoxon signed-rank tests
    pairs = [('XGBoost', 'PR'), ('XGBoost', 'ANN'), ('PR', 'ANN')]
    for model1, model2 in pairs:
        # Calculate differences to check for zero variance
        differences = data[model1] - data[model2]
        if np.all(differences == 0):
            stat = 0.0
            p = 1.0
        else:
            # Use zero_method='pratt' to handle zero differences
            stat, p = wilcoxon(data[model1], data[model2], zero_method='pratt')
        pairwise_results_list.append({
            'Metric': metric,
            'Comparison': f'{model1} vs {model2}',
            'Statistic': stat,
            'p-value': p
        })

# Create DataFrame for Friedman test results
friedman_results_df = pd.DataFrame(friedman_results)

print('\nFriedman Test Results:')
print(friedman_results_df)

# Create DataFrame for pairwise Wilcoxon test results
pairwise_results_df = pd.DataFrame(pairwise_results_list)

print('\nPairwise Wilcoxon Signed-Rank Test Results:')
print(pairwise_results_df)

# Save the DataFrames to CSV files
friedman_results_df.to_csv('friedman_test_results.csv', index=False)
pairwise_results_df.to_csv('pairwise_wilcoxon_results.csv', index=False)

# Optional: Export the DataFrames to Excel for easier viewing
with pd.ExcelWriter('statistical_analysis_results.xlsx') as writer:
    mean_metrics_df.to_excel(writer, sheet_name='Mean Metrics', index=False)
    friedman_results_df.to_excel(writer, sheet_name='Friedman Test', index=False)
    pairwise_results_df.to_excel(writer, sheet_name='Wilcoxon Tests', index=False)


Mean Performance Metrics:
           Metric     XGBoost          PR          ANN
0             mae    5.040339   11.521451     2.329645
1             mse   117.14172  296.260161    51.903281
2            rmse   10.823188   17.211468     7.201981
3              r2    0.873095    0.679392     0.943831
4   training_time  6098.33079  532.226326  4722.174172
5  inference_time    8.200664    9.572506     6.651212

Nemenyi post-hoc test results for mae:
          XGBoost        PR       ANN
XGBoost  1.000000  0.254114  0.254114
PR       0.254114  1.000000  0.004467
ANN      0.254114  0.004467  1.000000

Nemenyi post-hoc test results for mse:
          XGBoost        PR       ANN
XGBoost  1.000000  0.254114  0.254114
PR       0.254114  1.000000  0.004467
ANN      0.254114  0.004467  1.000000

Nemenyi post-hoc test results for rmse:
          XGBoost        PR       ANN
XGBoost  1.000000  0.254114  0.254114
PR       0.254114  1.000000  0.004467
ANN      0.254114  0.004467  1.000000

Nemenyi pos